In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_core.tracers.context import tracing_v2_enabled
from langchain.agents import AgentExecutor

In [4]:
search = TavilySearchResults()

In [5]:
search.description

'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.'

In [6]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

In [7]:
doc_chunks = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector_db = FAISS.from_documents(doc_chunks, OpenAIEmbeddings())
retriever = vector_db.as_retriever()

In [8]:
print(retriever.get_relevant_documents("how to upload a dataset")[0].page_content)

dataset uploading.Once we have a dataset, how can we use it to test changes to a prompt or chain? The most basic approach is to run the chain over the data points and visualize the outputs. Despite technological advancements, there still is no substitute for looking at outputs by eye. Currently, running the chain over the data points needs to be done client-side. The LangSmith client makes it easy to pull down a dataset and then run a chain over them, logging the results to a new project associated with the dataset. From there, you can review them. We've made it easy to assign feedback to runs and mark them as correct or incorrect directly in the web app, displaying aggregate statistics for each test project.We also make it easier to evaluate these runs. To that end, we've added a set of evaluators to the open-source LangChain library. These evaluators can be specified when initiating a test run and will evaluate the results once the test run completes. If we’re being honest, most of


In [9]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)
retriever_tool.description

'Search for information about LangSmith. For any questions about LangSmith, you must use this tool!'

In [10]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [11]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

```
[
    SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=[], 
            template='You are a helpful assistant'
        )
    ),
    MessagesPlaceholder(
        variable_name='chat_history', 
        optional=True
    ),
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=['input'], 
            template='{input}'
        )
    ),
    MessagesPlaceholder(variable_name='agent_scratchpad')
 ]
```

In [12]:
agent = create_openai_functions_agent(llm, [search, retriever_tool], prompt)

In [13]:
agent_executor = AgentExecutor(agent=agent, tools=[search, retriever_tool], verbose=True)  #type: ignore

In [14]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [15]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith testing'}`


LangSmith Overview and User Guide | 🦜️🛠️ LangSmith

applications can be expensive. LangSmith tracks the total token usage for a chain and the token usage of each step. This makes it easy to identify potentially costly parts of the chain.Collaborative debugging​In the past, sharing a faulty chain with a colleague for debugging was challenging when performed locally. With LangSmith, we've added a “Share” button that makes the chain and LLM runs accessible to anyone with the shared link.Collecting examples​Most of the time we go to debug, it's because something bad or unexpected outcome has happened in our application. These failures are valuable data points! By identifying how our chain can fail and monitoring these failures, we can test future chain versions against these known issues.Why is this so impactful? When building LLM applications, it’s often common to start without a 

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in the following ways:\n\n1. Tracking Token Usage: LangSmith can track the total token usage for a chain and the token usage of each step. This makes it easy to identify potentially costly parts of the chain during testing.\n\n2. Collaborative Debugging: LangSmith provides a "Share" button that allows you to share a faulty chain with a colleague for debugging. This makes it easier to collaborate and troubleshoot issues during testing.\n\n3. Collecting Examples: When testing an application, failures and unexpected outcomes are valuable data points. LangSmith allows you to identify how the chain can fail and monitor these failures. You can then test future chain versions against these known issues.\n\n4. Monitoring: Once your application is ready for production, LangSmith can be used to monitor your application. You can log all traces, visualize latency and token usage statistics, and troublesho

In [16]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.sfexaminer.com/news/climate/san-francisco-weather-forecast-calls-for-strongest-2024-rain/article_75347810-bfc3-11ee-abf6-e74c528e0583.html', 'content': "San Francisco is projected to receive 2.5 and 3 inches of rain, Clouser said, as well as gusts of wind up to 45 mph.  Bay Area starting Wednesday at 4 a.m. and a 24-hour wind advisory in San Francisco starting at the same time.  One of winter's 'stronger' storms to douse San Francisco  On the heels of record-breaking heat to open the week, heavy rain is slated to pound the Bay Area on Wednesday.Updated Feb 2, 2024 3.4 magnitude earthquake strikes near San Francisco Zoo Updated Feb 2, 2024 SF hopes to install 200 heat pumps in 200 days in at-risk homes Updated Feb 2, 2024 On the..."}, {'url': 'https://www.cnn.com/2024/02/04/us/california-atmospheric-river-flooding/index.html', 'content': 'f

{'input': 'whats the weather in sf?',
 'output': "The weather in San Francisco is currently experiencing heavy rain and strong winds. The city is projected to receive 2.5 to 3 inches of rain and gusts of wind up to 45 mph. There is a 24-hour wind advisory in effect. Please note that weather conditions can change rapidly, so it's always a good idea to check for the latest updates."}

In [18]:
with tracing_v2_enabled(project_name="Agent Quickstart"):
    agent_executor.invoke({"input": "Who is Avilay Parekh?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Avilay Parekh'}`


[{'url': 'https://www.linkedin.com/in/avilay', 'content': "I look forward to working with Avilay on other projects in the future!” Pranay Doshi  Avilay Parekh Sign in to view Avilay’s full profile Welcome back or  View Avilay’s full experience  View Avilay’s full profile Sign in Stay updated on your professional worldA cloud computing and machine learning expert who wants to build a world in which… | Learn more about Avilay Parekh's work experience, education,\xa0..."}, {'url': 'https://www.linkedin.com/posts/avilay_a-good-friend-of-mine-is-looking-for-opportunities-activity-7094464156149415936-P-02?trk=public_profile_like_view', 'content': 'Avilay Parekhâ€™s Post Machine Learning and Cloud Infrastructure  VP Engineering @ Productiv  Suresh is a great guy to work with. Great startup opportunity. DM me if you want more details. https://lnkd.in/gv8GkTP  Some more self-refle

In [19]:
with tracing_v2_enabled(project_name="Agent Quickstart"):
    response = agent_executor.invoke({"input": "Who won the Super Bowl this yaer?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Super Bowl winner 2021'}`


[{'url': 'https://www.cbsnews.com/news/super-bowl-winners-list-history/', 'content': 'Super Bowl 2024  Who won the 2023 Super Bowl last year?  third Super Bowl win.  49ers to see who will be crowned NFL champions for the 2023 season.54 minutes ago — 54 minutes ago2021: Tampa Bay Buccaneers — The Buccaneers dominated the Kansas City Chiefs in a 31-9 win, with Tom Brady, in his first season after leaving\xa0...'}, {'url': 'https://blog.ticketmaster.com/super-bowl-winners/', 'content': 'Super Bowl LV — Tampa Bay Buccaneers  Super Bowl LIII — New England Patriots  Super Bowl LIV — Kansas City Chiefs  Super Bowl LVI — Los Angeles Rams8 hours ago — 8 hours agoFor Kansas City, quarterback Patrick Mahomes — who led the team to their first victory in five decades in 2021— lost by double digits for the\xa0...'}, {'url': 'https://en.wikipedia.org/wiki/Super_Bowl_LV', 'conte

In [21]:
with tracing_v2_enabled(project_name="Agent Quickstart"):
    response = agent_executor.invoke({"input": "Who won the Super Bowl this year?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Super Bowl winner 2021'}`


[{'url': 'https://www.cbsnews.com/news/super-bowl-winners-list-history/', 'content': 'Super Bowl 2024  Who won the 2023 Super Bowl last year?  third Super Bowl win.  49ers to see who will be crowned NFL champions for the 2023 season.55 minutes ago — 55 minutes ago2021: Tampa Bay Buccaneers — The Buccaneers dominated the Kansas City Chiefs in a 31-9 win, with Tom Brady, in his first season after leaving\xa0...'}, {'url': 'https://blog.ticketmaster.com/super-bowl-winners/', 'content': 'Super Bowl LV — Tampa Bay Buccaneers  Super Bowl LIII — New England Patriots  Super Bowl LIV — Kansas City Chiefs  Super Bowl LVI — Los Angeles Rams8 hours ago — 8 hours agoFor Kansas City, quarterback Patrick Mahomes — who led the team to their first victory in five decades in 2021— lost by double digits for the\xa0...'}, {'url': 'https://en.wikipedia.org/wiki/Super_Bowl_LV', 'conte